This ipynb is for turnining your JSON into forceGraph suitable format, after you export it from the Gephi as a JSON file

The first cell is for fixing the the attribute keys, which will be broken once you export from the Gephi. It is important to have columns in your Gephi nodes page in this order:
- Id
- Label
- Interval
- #followers
- #friends
- name
- screen_name
- party


The second cell is for scaling the z values to some degree to make the 3D experience better

The third cell is for creating a smaller network by removing the nodes that contain follower less than the specified number of followers. We used this code for creating JSON file that are less than 25MB, so that we can upload the data to our GitHub Repo

In [1]:
import json

with open('politicianNodes.json', 'r') as f:
    data = json.load(f)

# Each has to contain an attribute called 'id' rather than 'key'
# Once the JSON file is exported from Gephi, the attributes are stored in a dictionary called 'attributes'
# Since the attributes are stored in a dictionary, we need to extract them and put them in the same level as the id
for node in data['nodes']:
    node['id'] = node.pop('key')
    node.update(node.pop('attributes'))

# To preserve the Network structure prepared in Gephi, we need to fix the x y z coordinates as fx fy fz
for node in data['nodes']:
    node['fx'] = node.pop('x')
    node['fy'] = node.pop('y')
    node['fz'] = node.pop('z')

    # Since the attribute names are broken during the export, we need to fix them
    node['num_followers'] = node.pop('0')
    node['num_following'] = node.pop('1')
    node['name'] = node.pop('2')
    node['screen_name'] = node.pop('3')
    node['party'] = node.pop('4')

    # Label attribute is not needed
    del node['label']

# The links are stored in a dictionary called 'edges'
# For forceGraph to work, we need to change the name to 'links'
data['links'] = data.pop('edges')

# Delete the attributes and options since they are not needed
del data['attributes']
del data['options']

# Export the data to a new JSON file
with open('politicianNodes_3D.json', 'w') as f:
    json.dump(data, f)

# If you would like to use this new JSON file as your default data
# go to forceGraph.js and change the line: let response = await fetch("./politicianNodes_S3DForce.json");
# to: let response = await fetch("./your_new_file_name.json");

In [2]:
import json

with open('politicianNodes_3D.json', 'r') as f:
    data = json.load(f)

for node in data['nodes']:
    node['fz'] *= 1000000

with open('politicianNodes_3D_Scaled.json', 'w') as f:
    json.dump(data, f)

# If you would like to use this new JSON file as your default data
# go to forceGraph.js and change the line: let response = await fetch("./politicianNodes_S3DForce.json");
# to: let response = await fetch("./your_new_file_name.json");

In [5]:
import json

with open('politicianNodes_3DForce.json') as f:
    data = json.load(f)

nodes = data['nodes']
links = data['links']
num_followers_less_than_100000 = 0

# Remove nodes with less than 100000 followers
for node in nodes:
    if node['num_followers'] < 100000:
        num_followers_less_than_100000 += 1
        node_id = node['id']
        links_to_remove = []
        for link in links:
            if link['source'] == node_id or link['target'] == node_id:
                links_to_remove.append(link)
        for link in links_to_remove:
            links.remove(link)
        nodes.remove(node)

new_data = {'nodes': nodes, 'links': links}

with open('politicianNodes_S3DForce.json', 'w') as f:
    json.dump(new_data, f)

print(len(nodes))
print(len(links))

527
138460
